In [1]:
import os
import pandas as pd
from PIL import Image

In [2]:
DIR_PRODUCT_CATALOG_NATIVE = 'data/faces_native'
DIR_PRODUCT_CATALOG_CONVERTED = 'data/faces_converted'
DIR_CSV_DATAFRAMES = 'csv_dataframes'
CSV_GTIN_DICT = f'{DIR_CSV_DATAFRAMES}/dictionary_gtin.csv'
CSV_INPUT_FACES = f'{DIR_CSV_DATAFRAMES}/input_faces.csv'

In [3]:
gtin_df = pd.read_csv(CSV_GTIN_DICT)
gtin_df = gtin_df.dropna()
gtin_df['GTIN'] = gtin_df['GTIN'].astype(int).astype(str)

In [4]:
os.makedirs(DIR_PRODUCT_CATALOG_CONVERTED, exist_ok=True)
for image_file in os.listdir(DIR_PRODUCT_CATALOG_NATIVE):
    image = Image.open(f'{DIR_PRODUCT_CATALOG_NATIVE}/{image_file}')
    image = image.convert('RGB')
    image.save(f'{DIR_PRODUCT_CATALOG_CONVERTED}/{image_file.split(".")[0]}.jpg', 'JPEG')

In [5]:
dict_for_pandas = {
    'image': [],
    'xmin': [],
    'ymin': [],
    'xmax': [],
    'ymax': [],
    'label': []
}

for image_file in os.listdir(DIR_PRODUCT_CATALOG_NATIVE):
    img = Image.open(f'{DIR_PRODUCT_CATALOG_NATIVE}/{image_file}')
    width, height = img.size

    name = image_file.split('.')[0]
    label = gtin_df.loc[gtin_df['GTIN'] == name]['Nazwa'].iloc[0]
    dict_for_pandas['image'].append(f'{name}.jpg')
    dict_for_pandas['xmin'].append(float(0))
    dict_for_pandas['ymin'].append(float(0))
    dict_for_pandas['xmax'].append(float(img.width))
    dict_for_pandas['ymax'].append(float(img.height))
    dict_for_pandas['label'].append(label)

In [6]:
df_faces = pd.DataFrame(dict_for_pandas)
for col in ['xmin', 'ymin', 'xmax', 'ymin']:
    df_faces[col] = df_faces[col].astype(float)
df_faces.to_csv(CSV_INPUT_FACES, index=False)